In [1]:
# IMPORT GENERIC PACKAGES
import numpy as np # numerical calc package
import pandas as pd # holds data
import matplotlib.pyplot as plt # plotting library
import seaborn as sns # pretty plotting
import pandas_profiling as pp
import datetime

In [2]:
sns.set(style='white', rc={'figure.figsize':(20,10)})

from sklearn.linear_model import LinearRegression # linear regression package
from sklearn.model_selection import train_test_split # split dataset
from sklearn.metrics import mean_squared_error as mse # Measurement metric

from sklearn.preprocessing import StandardScaler

In [3]:
import os
cwd = os.getcwd()
print(cwd)
os.chdir('C:/Users/mox.ballo/OneDrive/FTW/Lectures/week5/exercise/FTW3-Day5')

data = pd.read_csv('nyc-rolling-sales.csv', index_col=0)

C:\Users\mox.ballo\OneDrive\FTW\Lectures\week5\exercise\FTW3-Day5


In [4]:
##Data Transformation
data['SALE PRICE'] = pd.to_numeric(data['SALE PRICE'], errors = 'coerce')
data['LAND SQUARE FEET'] = pd.to_numeric(data['LAND SQUARE FEET'], errors = 'coerce')
data['GROSS SQUARE FEET'] = pd.to_numeric(data['GROSS SQUARE FEET'], errors = 'coerce')
data['SALE DATE'] = pd.to_datetime(data['SALE DATE'], errors = 'coerce')
data['YEAR'] = pd.DatetimeIndex(data['SALE DATE']).year

data.info() #check 'SALE PRICE', LAND SQUARE FEET, and GROSS SQUARE FEET if float

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84548 entries, 4 to 8413
Data columns (total 22 columns):
BOROUGH                           84548 non-null int64
NEIGHBORHOOD                      84548 non-null object
BUILDING CLASS CATEGORY           84548 non-null object
TAX CLASS AT PRESENT              84548 non-null object
BLOCK                             84548 non-null int64
LOT                               84548 non-null int64
EASE-MENT                         84548 non-null object
BUILDING CLASS AT PRESENT         84548 non-null object
ADDRESS                           84548 non-null object
APARTMENT NUMBER                  84548 non-null object
ZIP CODE                          84548 non-null int64
RESIDENTIAL UNITS                 84548 non-null int64
COMMERCIAL UNITS                  84548 non-null int64
TOTAL UNITS                       84548 non-null int64
LAND SQUARE FEET                  58296 non-null float64
GROSS SQUARE FEET                 56936 non-null float64
Y

In [5]:
##Data Cleaning
data1 = data[data['SALE PRICE']>0] 
sales95 = np.percentile(data1['SALE PRICE'].fillna(0),95) 
sales05 = np.percentile(data1['SALE PRICE'].fillna(0),5) 
data1['SALE PRICE'].describe()


count    5.975900e+04
mean     1.494927e+06
std      1.232953e+07
min      1.000000e+00
25%      3.700000e+05
50%      6.255000e+05
75%      1.070000e+06
max      2.210000e+09
Name: SALE PRICE, dtype: float64

In [6]:
##Remove outliers of sale price
datareg = data1[data1['SALE PRICE']>sales05]
datareg = data1[data1['SALE PRICE']<sales95]
datareg['SALE PRICE'].describe()

count    5.676700e+04
mean     7.875704e+05
std      6.827964e+05
min      1.000000e+00
25%      3.550000e+05
50%      6.000000e+05
75%      9.600000e+05
max      3.999999e+06
Name: SALE PRICE, dtype: float64

In [7]:
neigh = pd.get_dummies(datareg['NEIGHBORHOOD'])
blcltos = pd.get_dummies(datareg['BUILDING CLASS AT TIME OF SALE'])
taxcl = pd.get_dummies(datareg['TAX CLASS AT PRESENT'])
#ease = pd.get_dummies(datareg['EASE-MENT'])
blclpres = pd.get_dummies(datareg['BUILDING CLASS AT PRESENT'])
#add = pd.get_dummies(datareg['ADDRESS'])
#apt = pd.get_dummies(datareg['APARTMENT NUMBER'])
##Removed variables that I think that are not relevant, also kulang memory ko hehe

In [8]:
dummies = list(neigh.columns.values) + list(blcltos.columns.values) + list(taxcl.columns.values) + list(blclpres.columns.values) 

In [9]:
datareg = pd.concat([datareg, neigh,blcltos,taxcl,blclpres], axis=1, sort=False)

In [10]:
features = dummies + ['GROSS SQUARE FEET','LAND SQUARE FEET','LOT','BOROUGH','BLOCK','LOT','ZIP CODE','RESIDENTIAL UNITS','COMMERCIAL UNITS','TOTAL UNITS','YEAR BUILT','TAX CLASS AT TIME OF SALE','YEAR']

In [11]:
X = datareg[features].fillna(0)
y = datareg['SALE PRICE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 42)

In [12]:
from sklearn.neural_network import MLPRegressor

MLPReg = MLPRegressor(random_state=0)
MLPReg = MLPReg.fit(X_train, y_train)
y_predicted = MLPReg.predict(X_test)
np.sqrt(mse(y_predicted, y_test))

MemoryError: 